In [1]:
import pandas as pd
import itertools
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv('KaggleCovidDataset.csv').applymap(lambda x : 1 if x == "Yes" else 0)
df

/tmp/ipykernel_23968/3842546721.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = pd.read_csv('KaggleCovidDataset.csv').applymap(lambda x : 1 if x == "Yes" else 0)


,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,1,1,1,1,1,0,0,0,0,1,...,1,1,0,1,0,1,1,0,0,1
1,1,1,1,1,0,1,1,1,0,0,...,1,0,0,0,1,1,0,0,0,1
2,1,1,1,1,1,1,1,1,0,1,...,1,1,1,0,0,0,0,0,0,1
3,1,1,1,0,0,1,0,0,1,1,...,0,0,1,0,1,1,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,1,1,0,1,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
5430,1,1,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
5431,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5432,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
def aprioriBAD(df, min_support, use_colnames):
    df = df.copy(deep=True)
    if not use_colnames:
        df.columns = [idx for idx in range(len(df.columns))]

    # Generate 1-itemsets
    oneItemSets = df.apply(lambda x: x.value_counts(normalize=True).get(1, 0)).loc[lambda x: x >= min_support].to_dict()
    oneItemSets = {frozenset((k,)): v for k, v in oneItemSets.items()}

    # Function to generate k-itemsets
    def generate_k_itemset(itemset, k):
        itemset_k = []
        for item1 in itemset:
            for item2 in itemset:
                if item1[:-2] == item2[:-2] and item1[-2:] != item2[-2:]:
                    new_item = tuple(sorted(set(item1 + item2)))
                    if len(new_item) == k:
                        itemset_k.append(new_item)
        itemset_k = list(set(itemset_k))
        itemset_k_support_map = {}
        for kitem in itemset_k:
            countPresent = 0
            for doc in range(df.shape[0]):
                addCount = True
                for item in kitem:
                    if df[item][doc] == 0:
                        addCount = False
                        break
                if addCount:
                    countPresent += 1
            support = countPresent/df.shape[0]
            if support >= min_support:
                itemset_k_support_map[frozenset(kitem)] = support
        return itemset_k_support_map
    
    # Create all itemsets
    allItemSets = {**oneItemSets}
    
    # Generate k-itemsets for k > 1
    kItemSets = oneItemSets
    k = 2
    while len(kItemSets) > 0:
        kItemSets = generate_k_itemset([tuple(key) for key in kItemSets.keys()], k)
        allItemSets.update(kItemSets)
        k += 1

    return pd.DataFrame([(v, k) for k, v in allItemSets.items()], columns=['support', 'itemsets'])

def association_rulesBAD(df, min_confidence):
    frequent_itemsets = {frozenset(itemset): support for itemset, support in zip(df['itemsets'], df['support'])}

    rules = []
    for itemset in frequent_itemsets:
        n = len(itemset)
        if n > 1:
            for i in range(1, n):
                lefts = list(itertools.combinations(itemset, i))
                for left in lefts:
                    left = frozenset(left)
                    right = itemset.difference(left)
                    confidence = frequent_itemsets[itemset] / frequent_itemsets[left]
                    if confidence >= min_confidence:
                        rules.append((left, right, confidence))
                        
    return pd.DataFrame(rules, columns=['antecedents', 'consequents', 'confidence'])

In [4]:
aBadDF = aprioriBAD(df, 0.6, True)
aDF = apriori(df, 0.6, True)

/home/jass/.local/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [5]:
good, bad = set(aDF['itemsets']), set(aBadDF['itemsets'])
whatsnotthere = good.difference(bad)
whatsnotthere

set()

In [6]:
association_rulesBAD(aBadDF, min_confidence=0.6)

,antecedents,consequents,confidence
0,(Dry Cough),(Fever),0.813095
1,(Fever),(Dry Cough),0.819565
2,(Sore throat),(Fever),0.867189
3,(Fever),(Sore throat),0.802247
4,(COVID-19),(Breathing Problem),0.768652
5,(Breathing Problem),(COVID-19),0.930663
6,(Sore throat),(Dry Cough),0.845687
7,(Dry Cough),(Sore throat),0.776178
8,(COVID-19),(Dry Cough),0.884782
9,(Dry Cough),(COVID-19),0.900395


In [7]:
association_rules(aDF, metric='confidence', min_threshold=0.6)[['antecedents', 'consequents', 'confidence']]

,antecedents,consequents,confidence
0,(COVID-19),(Breathing Problem),0.768652
1,(Breathing Problem),(COVID-19),0.930663
2,(Fever),(Dry Cough),0.819565
3,(Dry Cough),(Fever),0.813095
4,(Sore throat),(Fever),0.867189
5,(Fever),(Sore throat),0.802247
6,(COVID-19),(Fever),0.857175
7,(Fever),(COVID-19),0.879242
8,(Sore throat),(Dry Cough),0.845687
9,(Dry Cough),(Sore throat),0.776178
